In [2]:
import pandas as pd
from yaml import safe_load
from pandas import json_normalize

from sentence_transformers import SentenceTransformer
from transformers import AutoModel, BertTokenizer
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import numpy as np

In [3]:
def calculating_similarity_text_df(definition, text_embeddings):
    definition_embedding= model.encode(definition, batch_size = 8, show_progress_bar = True)
    similarity_def=cosine_similarity( [definition_embedding], text_embeddings)
    similarity_def_sorted = similarity_def.argsort()
    id_1 = []
    id_2 = []
    score = []
    for index,array in enumerate(similarity_def_sorted):
        id_1.append(index)
        score.append(similarity_def[index][array[-1]])
        id_2.append(array[-1])
    index_df = pd.DataFrame({'id_1' : id_1,
                          'id_2' : id_2,
                          'score' : score})
    return index_df

In [4]:
def calculating_similarity_text(definition, text_embeddings):
    definition_embedding= model.encode(definition, batch_size = 8, show_progress_bar = True)
    similarities=cosine_similarity( [definition_embedding], text_embeddings)
    similarities_sorted = similarities.argsort()
    results={}
    for i in range(-1, -12, -1):
        similar_index=similarities_sorted[0][i]
        rank=similarities[0][similar_index]
        results[uris[similar_index]]=[terms_info[similar_index][1],terms_info[similar_index][2], terms_info[similar_index][4], terms_info[similar_index][0], documents[similar_index], rank]
    return results

In [5]:
def get_document(uri):
    uri="<"+uri+">"
    sparql = SPARQLWrapper("http://localhost:3030/edition7Clean/sparql")
    query="""
    PREFIX eb: <https://w3id.org/eb#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?definition ?term
        WHERE {{
            %s a eb:Article ;
               eb:name ?term ;
               eb:definition ?definition . 
            }
            UNION {
            %s a eb:Topic ;
              eb:name ?term ; 
              eb:definition ?definition . 
            }
       } 
    """ %(uri, uri)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    term = results["results"]["bindings"][0]["term"]["value"]
    definition=results["results"]["bindings"][0]["definition"]["value"]
    return term, definition